In [179]:
#Basic Imports
import pandas as pd
import numpy as np
import matplotlib as plt
import statsmodels.api as sm
import seaborn as sns
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,summarize)
from sklearn.model_selection import train_test_split

In [180]:
#Classifying Imports
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder

In [181]:
Default = load_data("Default")
Default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [182]:
enc = OrdinalEncoder(categories = [["No","Yes"]])
Default["student_encoded"] = enc.fit_transform(Default[["student"]])

In [183]:
default_train,default_test = train_test_split(Default,test_size=2000)

In [184]:
allvars = Default.columns.drop(["student","default"])
modelspec = MS(allvars)
X_train  = modelspec.fit_transform(default_train)
Y_train = default_train.default == "Yes"
model = sm.GLM(Y_train,X_train,family=sm.families.Binomial())
results = model.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-10.546500,0.540000,-19.543,0.000
balance,0.005700,0.000000,22.166,0.000
income,-0.000002,0.000009,-0.207,0.836
student_encoded,-0.830300,0.261000,-3.177,0.001


In [185]:

x_test = modelspec.fit_transform(default_test)
probs = results.predict(exog =  x_test)

In [186]:
labels = np.array(["No"]*2000)
labels[probs > 0.5] = "Yes"
labels = np.char.replace(labels,"Ye","Yes")
confusion_table(labels,default_test.default)

Truth,No,Yes
Predicted,,
No,1925,43
Yes,8,24


In [187]:
error = (45+12)/2000 * 100
print(f"Validation Set Test Error Rate {error}")

Validation Set Test Error Rate 2.85


We got an initial Validation Set Test Error Rate of 2.85% without the use of Student as a predictor. When we added it, the test error remained the same. 

Using Summarize, we saw that our standard errors for balance and income were pretty much 0 and 9x10e^-6. We'll now attempt to compute the same values using the bootstrap